In [86]:
import numpy as np
import gmsh

gmsh.initialize() #inicia Gmsh
gmsh.model.add('PROBLEMA2') #genera el modelo


# Generado de la figura
L1 = 20
L2 = 10
#Lc = 20 #caso del ejercicio
Lc = 1
Lcr = 1  #Refina aun mas el mallado cerca del circulo

#chapa
p1 = gmsh.model.geo.addPoint(0, 0, 0, Lc)
p2 = gmsh.model.geo.addPoint(0, L2, 0, Lc) 
p3 = gmsh.model.geo.addPoint(L1, L2, 0, Lc) 
p4 = gmsh.model.geo.addPoint(L1, 0, 0, Lc)

#circulo
p5 = gmsh.model.geo.addPoint(9,L2/2,0,Lcr)
p6 = gmsh.model.geo.addPoint(11,L2/2,0,Lcr)
Pcentro = gmsh.model.geo.addPoint(10,L2/2,0,Lcr)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

semi1 = gmsh.model.geo.addCircleArc(p5,Pcentro ,p6) #genera semicirculos , segun como acomode p5 y p6 ira por arriba o por abajo
semi2 = gmsh.model.geo.addCircleArc(p6,Pcentro, p5)

C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
C2 = gmsh.model.geo.addCurveLoop([semi1, semi2])

S1 = gmsh.model.geo.addPlaneSurface([C1,C2]) #asi se genera la superifice descontando el circulo

gmsh.model.geo.synchronize()


#Generado de las condiciones de contorno

TraccionadoIzq = gmsh.model.addPhysicalGroup(1, [l1]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,TraccionadoIzq,'TraccionadoIzq')

TraccionadoDer = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1,TraccionadoDer,'TraccionadoDer')

Superficie = gmsh.model.addPhysicalGroup(2,[S1]) #2 es para superficies!
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')




gmsh.model.mesh.generate(2)

In [87]:
NodeInfo = gmsh.model.mesh.get_nodes()
NodeInfo[1].shape
NumeroNodos = NodeInfo[0].shape[0]
MN = NodeInfo[1].reshape(NumeroNodos , 3)


ElementInfo = gmsh.model.mesh.get_elements()
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])


In [88]:
#TraccionadosIzq
NodosTraccionadosIzq = gmsh.model.mesh.get_nodes_for_physical_group(1,TraccionadoIzq)
A = NodosTraccionadosIzq[0].reshape(-1,1)
B = np.zeros((len(A),2))
MNTI = np.concatenate((A,B) , axis = 1)
MNTI[:,1] = 5000

#TraccionadosDer
NodosTraccionadosDer = gmsh.model.mesh.get_nodes_for_physical_group(1, TraccionadoDer)
C = NodosTraccionadosDer[0].reshape(-1,1)
D = np.zeros((len(A),2))
MNTD = np.concatenate((C,D) , axis = 1)
MNTD[:,1] = 5000


Nodoslibres = []

Nodos = gmsh.model.mesh.get_nodes_for_physical_group(2, Superficie)
for i in Nodos[0]:
    if i in NodosTraccionadosDer[0]:
        l = 0
    elif i in NodosTraccionadosIzq[0]:
        l = 0
    else:
        Nodoslibres.append(i)
        
E = np.array(Nodoslibres).reshape(-1,1)
D = np.zeros((len(E),2))
Interior = np.concatenate((E,D) , axis = 1)


Fuerzas_mallado = np.concatenate((MNTI,Interior,MNTD) , axis = 0)



In [89]:
gmsh.fltk.run()
gmsh.finalize()

In [80]:
espesor = 1
E = 30E6
v = 0.3

D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []

In [81]:
Fuerza = np.zeros(len(MN)*2)
for i in Fuerzas_mallado:
    Nodo,valorx,valory = i 
    nodo = int(Nodo-1)
    Fuerza[2*nodo] = valorx
    Fuerza[2*nodo+1] = valory

fuerza = Fuerza.reshape(-1,1)

In [82]:
for i in range (len(MC)):
    
    I,M,J = MC[i]    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)
    
    x = np.array([MN[i,0],MN[m,0],MN[j,0]])
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])
    Bm = y[0] - y[2] ;   Bi = y[2] - y[1] ;    Bj = y[1] - y[0]
    Gi = x[1] - x[2] ;   Gm = x[2] - x[0] ;    Gj = x[0] - x[1]  
    
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    Area = abs(np.linalg.det(Matriz_coordenadas)/2)

    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bj,0,Bm,0],[0,Gi,0,Gj,0,Gm],[Gi,Bi,Gj,Bj,Gm,Bm]])

    Matrices_Bloc.append(Matriz1)
    Matriz_local = espesor * Area * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,j*2,j*2+1,m*2,m*2+1]) #aca pasa algo raro
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    

[[ 940.62057796 -506.79160632    0.         ...    0.
     0.            0.        ]
 [-506.79160632 1721.03770698    0.         ...    0.
     0.            0.        ]
 [   0.            0.          601.36925498 ...    0.
     0.            0.        ]
 ...
 [   0.            0.            0.         ... 4831.64745838
     0.            0.        ]
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 [   0.            0.            0.         ...    0.
     0.            0.        ]]


In [90]:
Desplazamientos = np.linalg.solve(Matriz_global,Fuerzas)

ValueError: solve: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m,n)->(m,n) (size 312 is different from 626)